In [1]:
import time
from selenium import webdriver
import re
import pandas as pd
import time, random
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd
###

import requests
from bs4 import BeautifulSoup

In [43]:
df = pd.read_excel("c:\import\ssss.xlsx")




In [44]:
# 0. float으로 되어있는거가 228 인데 228.0 으로 되어있어서 228로 다시 바꿔주는 함수
def dotzero(mt) :
    mt = float(mt)
    if mt.is_integer() :
        mt = int(mt)
    else :
        mt = mt
    return mt



In [45]:
# 1. 키워드 뽑아내는 작업
search_keyword = []

for i in range(len(df)) :
    
    if df['lc_pk'][i] == 1 : #################################################### 사료
    
        if df['sell_unit'][i] == 1 :
            number = df['weight'][i] * df['gift_unit'][i] 
            ###
            number = dotzero(number)
            ###
            result = str(number) + df['weight_unit'][i]
            
            if df['weight_unit'][i] == 'g' and number >= 1000 :
                num = number/1000
                ####
                num = dotzero(num)
                ####
                add = str(num) + 'kg'
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],add]
                search_keyword.append(going_in)
            
            else :
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],result]
                search_keyword.append(going_in)
            
        else :
            first_number = df['weight'][i] * df['gift_unit'][i] * df['sell_unit'][i]
            ####
            first_number = dotzero(first_number)
            ####
            if df['weight_unit'][i] == 'g' and first_number >= 1000 :
                result1 = str(first_number/1000)+ 'kg' 
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],result1]
                search_keyword.append(going_in)
                
            else :
                result1 = str(first_number) + df['weight_unit'][i] 
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],result1]
                search_keyword.append(going_in)
            
            
            second_number = df['weight'][i] * df['gift_unit'][i]
            ####
            second_number = dotzero(second_number)
            ####
            if df['weight_unit'][i] == 'g' and second_number >= 1000 :
                sec_num = second_number/1000
                ####
                sec_num = dotzero(sec_num)
                ####
                result2 = str(sec_num)+ 'kg' + '"' + ' ' +  '"' +str(df['sell_unit'][i])
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],result2]
                search_keyword.append(going_in)
                
            else :
                result2 = str(second_number) + df['weight_unit'][i] + '"' + ' ' +  '"' + str(df['sell_unit'][i])
                going_in = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],result2]
                search_keyword.append(going_in)
            
    else : ##################################################################### 간식 & 영양제
        
        if df['weight'][i] != 1 :
            
            first_number = df['weight'][i] * df['gift_unit'][i] * df['sell_unit'][i]
            ####
            first_number = dotzero(first_number)
            ####
            result1 = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],str(first_number)]
            search_keyword.append(result1)
            
            second_number = df['gift_unit'][i] * df['sell_unit'][i]
            ####
            second_number = dotzero(second_number)
            ####
            result2 = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],str(second_number)]
            search_keyword.append(result2)
            
        else :
            
            number = df['gift_unit'][i] * df['sell_unit'][i]
            ####
            number = dotzero(number)
            ####
            result = [df['product_id'][i],df['product_name'][i],df['weight'][i], df['weight_unit'][i], df['gift_unit'][i], df['sell_unit'][i],str(second_number)]
            search_keyword.append(result)
    

In [46]:
column = ['product_id','product_name','weight','weight_unit','gift_unit',"sell_unit","keyword"]
pre_final = pd.DataFrame.from_records( search_keyword , columns = column )

# final
# final.to_excel("hehe2.xlsx")

In [47]:
# 2. 검색해서 링크 가져오는 작업 (끝나고 액셀에서 모든열 기준 중복제거 해야됨 - 예를들어 200g 이랑 100g 2개 두번 검색해서 겹치는 결과가 있을 수 있어서)

#들어가야하는것
#결과창 안나오면....?
result = []
for i in range(len(pre_final)) :
    
    driver = webdriver.Chrome('./chromedriver95.exe')
    key = pre_final['product_name'][i] + ' ' + '"' + pre_final['keyword'][i] + '"'
    driver.get("https://search.shopping.naver.com/search/all?query={}".format(key))
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.randint(4, 6))
    
    큰박스 = driver.find_elements_by_css_selector('li[class="basicList_item__2XT81"]')
    
    for ak in 큰박스 :
        try :
            
            오류 = ak.find_elements_by_css_selector('a[class="basicList_compare__3AjuT"]')
            오류[0]
            
        except :
            
            최종택배비 = ak.find_elements_by_css_selector('em[class="basicList_option__3eF2s"]')[0].text
            최종택배비 = 최종택배비.replace("배송비","")
            최종택배비 = 최종택배비.strip()
            link_name = ak.find_element_by_class_name("basicList_link__1MaTN").text
            가격 = ak.find_element_by_css_selector('span[class="price_num__2WUXn"]').text
            link_address = ak.find_elements_by_class_name("basicList_link__1MaTN")[0].get_attribute('href')
            pre_result = [pre_final['product_id'][i],pre_final['product_name'][i],pre_final['weight'][i],pre_final['weight_unit'][i],pre_final['gift_unit'][i],pre_final['sell_unit'][i],link_name,link_address,최종택배비,가격]
            result.append(pre_result)
            
        else : #그 이후에 일반적으로 진행
            pass
        
    driver.quit()


In [42]:
column = ['product_id','product_name','weight','weight_unit','gift_unit',"sell_unit","link_name",'link_address','택배비','가격']
final = pd.DataFrame.from_records( result , columns = column )

final
final.to_excel("tet.xlsx")